### This is an extended version of the previous code. Users can specify all the movies they want info about in the movies list given in the below cell. This block of code will not only print the information simultaneously but you can also get the data in tabular form also.

In [3]:
movies= ['Pursuit of Happyness','Shawshank Redemption','Andhadhun','Dil Chahta Hai']


from bs4 import BeautifulSoup
import requests

titlelist=[]
typelist=[]
durationlist=[]
imdb_ratinglist=[]
votelist=[]
metascorelist=[]
castlist=[]
directorlist=[]
countrylist=[]
languagelist=[]
rdatelist=[]
genrelist=[]
certificatelist=[] 
storylist=[]



for input in movies:
    response = requests.get('http://www.imdb.com/find?ref_=nv_sr_fn&q=' +input+ ' &s=tt')
    soup = BeautifulSoup(response.content, 'lxml')
    # Fetching the movie id so that the browser can be directed to that webpage. Note out of many searches we are taking the 1st search result
    table = soup.find('table',class_='findList')
    movieid = table.tr.a['href']
    movielink = "http://www.imdb.com" + movieid
    moviepage = requests.get(movielink)
    soup2 = BeautifulSoup(moviepage.content, 'lxml')
    
    print('\n')
    print('\n')
    print('\n')
    print('\n')
    
    #title of movie/show
    uncleaned = soup2.find('div', class_ = 'title_wrapper').h1.text
    #it contains some weird text and extra &nbsp at end
    uncleaned =uncleaned.replace('\xa0','')
    title = uncleaned.strip()
    print("Title is - {}".format(title))
    titlelist.append(title)

    # Type, Duration/Seasons
    durncounter=1
    wrapper = soup2.select_one('.title_wrapper .subtext')
    links = wrapper.find_all('a')
    links
    for i in range(0,len(links)):
        if 'releaseinfo' in links[i]['href']:
            value = links[i].string.strip()
    if value.startswith('TV '):
        type = 'TV Series'
        typelist.append(type)
        seasons = soup2.select('.seasons-and-year-nav a')[0].string
        duration = seasons+" seasons"
    else:
        type='Movie'
        typelist.append(type)
        try:
            duration = wrapper.find('time').string.strip()
        except Exception as e:
            durncounter=0


    if(type == 'Movie') and durncounter!= 0:
        print("\nThe duration is", duration)
        durationlist.append(duration)
    elif(type == 'TV Series'):
        print("\nThe number of seasons are ", duration)
        durationlist.append(duration)
    else:
        print("\nDuration is not given on imdb page")
        durationlist.append(duration)

    #imdb rating
    imdbcounter=1
    try:
        imdb_rating = float(soup2.select('.ratingValue span')[0].text)
    except Exception as e:
        imdbcounter=0
    if(imdbcounter !=0):
        print("\nThe imdb rating of the {} is = {}".format(type,imdb_rating))
        imdb_ratinglist.append(imdb_rating)
    else:
        print("\nimdb rating is not given on imdb page")
        imdb_ratinglist.append(imdb_rating)



    #Total number of votes
    votecounter=1
    try:
        votes = soup2.find('span', {'itemprop':'ratingCount'}).string
        votes = votes.replace(',',"")
        votes = int(votes)
    except Exception as e:
        votecounter=0
    if(votecounter!=0):
        print("\nThe imdb rating is calculating on the basis of {} mumber of votes".format( votes))
        votelist.append(votes)
    else:
        print("\nNumber of votes are not given on imdb page")
        votelist.append(votes)

    #metascore    
    try:
        metascore = float(soup2.select('.metacriticScore span')[0].text) if float(soup2.select('.metacriticScore span')[0].text) else None
        print("\nMetascore - ",metascore)
        metascorelist.append(metascore)
    except Exception as e:
        metascore=None
        print("\nMetascore is not available in imdb site")
        metascorelist.append(metascore)

    # cast values
    namelist=[]
    oddnames=[]
    evennames =[]
    ttag = soup2.find_all('tr', class_= 'odd')
    for i in ttag:
        namelist.append( i.find('td', attrs={'class': None}))

    for i in namelist:
        if (i.a) is None:
            pass
        else:
            oddnames.append(i.a.string.strip())

    ttag = soup2.find_all('tr', class_= 'even')
    namelist=[]

    for i in ttag:
        namelist.append( i.find('td', attrs={'class': None}))

    for i in namelist:
        if (i.a) is None:
            pass
        else:
            evennames.append(i.a.string.strip())

    cast = oddnames+evennames
    print("\nCast is as follows",cast)
    castlist.append(cast)

    # Extracting directors
    directors=[]
    director_container = soup2.find('div',class_ = 'credit_summary_item')

    if((director_container.find('h4').string =='Director:') or (director_container.find('h4').string =='Creator:') or
    (director_container.find('h4').string =='Creators:') or (director_container.find('h4').string =='Directors:')) :
        directors_container = director_container.find_all('a')
        for i in directors_container:
            directors.append(i.string)
        print("\nDirectors are", directors)
        directorlist.append(directors)
    else:
        print("\nDirector information not given in imdb site")
        directorlist.append(directors)


    # Country of production
    country=[]
    titledetails = soup2.find('div',id='titleDetails')
    titleheading = titledetails.find_all('a')
    for i in range(0,len(titleheading)):
        if('country_of_origin' in titleheading[i]['href']):
            country.append(titleheading[i].string)
    print("\nProduction Countries-", country)
    countrylist.append(country)

    # Language
    languages=[]
    titledetails2 = soup2.find('div',id='titleDetails')
    titleheading2 = titledetails2.find_all('a')
    for i in range(0,len(titleheading2)):
        if('primary_language' in titleheading2[i]['href']):
            languages.append(titleheading2[i].string)
    print("\n{} is available in Languages- {}".format(type, languages))
    languagelist.append(languages)

    # release date
    dateheading = soup2.select('.txt-block h4')
    for i in range(0,len(dateheading)):
        if(dateheading[i].string=='Release Date:'):
            releasedate = dateheading[i].next_sibling.strip()
    print("\n{} was released in- {}".format(type, releasedate))
    rdatelist.append(releasedate)


    # genres
    genres=[]
    wrapper = soup2.select_one('.title_wrapper .subtext')
    links = wrapper.find_all('a')
    for i in links:
        if 'genres' in i['href']:
            genres.append(i.string)
    print("\nFollowing are the genres of the {} - {}".format(type,genres))
    genrelist.append(genres)

    # Certificate
    wrapper = soup2.select_one('.title_wrapper .subtext')
    certificate = wrapper.next_element.strip()
    if(certificate != ''):
        print("\nThe {} has been given {} certificate".format(type,certificate))
        certificatelist.append(certificate)
    else:
        print("\nCertificate details are not given on the imdb page")
        certificatelist.append(certificate)


    # Storyline
    # Here we have links inside story for some movies. So normal .strings wont work
    story = soup2.select_one('#titleStoryLine div span').get_text().strip()
    if(story!=''):
        print("\nHere is a description of the storyline - \n", story)
        storylist.append(story)
    else:
        print("\nStory description is not given on imdb page")
        storylist.append(story)











Title is - Pursuit of Happyness(2005)

The duration is 1h

The imdb rating of the Movie is = 7.5

The imdb rating is calculating on the basis of 61 mumber of votes

Metascore is not available in imdb site

Cast is as follows ['Frank Navin', 'Heather Kantor', 'Lawrence Steele', 'John Navin', 'Cynthia Plaster Caster', 'Fabio Zambernardi']

Directors are ['Patrick McGuinn']

Production Countries- ['USA']

Movie is available in Languages- ['English']

Movie was released in- 16 July 2005 (USA)

Following are the genres of the Movie - ['Documentary']

Certificate details are not given on the imdb page

Here is a description of the storyline - 
 If you haven't yet heard of The Aluminum Group - the critically adored Chicago pop band fronted by gay brothers John and Frank Navin - make sure to see this musical celebration of their past and present.








Title is - Shawshank Redemption

Duration is not given on imdb page

The imdb rating of the Movie is = 8.3

The imdb rating is calcul

In [4]:
import pandas as pd


d= {'title':titlelist,'type':typelist,'duration':durationlist,'imdb_rating':imdb_ratinglist,'votelist':votes,
    'metascore':metascorelist,'cast':castlist,'director':directorlist,'country':countrylist,'language':languagelist,
    'rdate':rdatelist,'genre':genrelist,'certificate':certificatelist,'story':storylist}

results= pd.DataFrame(d)
results

,title,type,duration,imdb_rating,votelist,metascore,cast,director,country,language,rdate,genre,certificate,story
0,Pursuit of Happyness(2005),Movie,1h,7.5,64679,None,"[Frank Navin, Heather Kantor, Lawrence Steele,...",[Patrick McGuinn],[USA],[English],16 July 2005 (USA),[Documentary],,If you haven't yet heard of The Aluminum Group...
1,Shawshank Redemption,Movie,1h,8.3,64679,None,"[Amanda Jaros, Bob Turton, Mackenzie Marsh]","[Sunday Boling, Michael L. Kuhn]",[],[],4 July 2013 (USA),"[Short, Comedy]",,|
2,Andhadhun(2018),Movie,2h 19min,8.3,64679,None,"[Ayushmann Khurrana, Radhika Apte, Manav Vij, ...",[Sriram Raghavan],[India],"[Hindi, English]",5 October 2018 (India),"[Crime, Thriller]",UA,Akash a blind pianist is preparing for a Londo...
3,Dil Chahta Hai(2001),Movie,3h 3min,8.1,64679,None,"[Aamir Khan, Akshaye Khanna, Sonali Kulkarni, ...",[Farhan Akhtar],[India],[Hindi],10 August 2001 (India),"[Comedy, Drama, Romance]",Unrated,"Three young men Akash, Sameer, and Siddharth a..."


In [52]:
results.to_csv('movies.csv')